<a href="https://colab.research.google.com/github/iguv221/Tobigs_Assignment/blob/main/11_AE_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [44]:
## google colab 을 사용했기 때문에 drive mount 을 함

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [45]:
## 필요 클래스 임포트

import random
import pandas as pd
import numpy as np
import os
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from tqdm.auto import tqdm
from sklearn.metrics import f1_score
import warnings
warnings.filterwarnings("ignore")

In [46]:
## 구글 코렙에서는 무료로 GPU 제한적으로 사용 가능하기 때문에 device 에 미리 세팅을 합니다.

GPU_NUM = 0
device = torch.device(f'cuda:{GPU_NUM}' if torch.cuda.is_available else 'cpu')
torch.cuda.set_device(device)

print(torch.cuda.get_device_name(0))
print(torch.cuda.is_available())
print(torch.__version__)

Tesla T4
True
1.12.1+cu113


In [47]:
## Train data 을 받아들입니다.
## 신용카드와 관련된 데이터이지만 일반적인 정형 데이터와 다르게 각 column 에 대한 설명이나 특징이 없습니다.

Train_Data = pd.read_csv('/content/drive/MyDrive/ToBigs/첫 데이콘/Data/train.csv')
Train_Data = Train_Data.drop(columns=['ID'])
Train_Data.head()

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,V21,V22,V23,V24,V25,V26,V27,V28,V29,V30
0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,0.207643,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,4.983721,-0.994972
1,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,-0.054952,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,1.418291,-0.994972
2,-0.425966,0.960523,1.141109,-0.168252,0.420987,-0.029728,0.476201,0.260314,-0.568671,-0.371407,...,-0.208254,-0.559825,-0.026398,-0.371427,-0.232794,0.105915,0.253844,0.081080,-0.256131,-0.994960
3,-0.644269,1.417964,1.074380,-0.492199,0.948934,0.428118,1.120631,-3.807864,0.615375,1.249376,...,1.943465,-1.015455,0.057504,-0.649709,-0.415267,-0.051634,-1.206921,-1.085339,0.262698,-0.994901
4,-0.894286,0.286157,-0.113192,-0.271526,2.669599,3.721818,0.370145,0.851084,-0.392048,-0.410430,...,-0.073425,-0.268092,-0.204233,1.011592,0.373205,-0.384157,0.011747,0.142404,0.994900,-0.994901


In [48]:
## Valid data 을 받아들입니다.
## Valid data 은 train data 에 과적합되는 것을 방지하기 위한 데이터셋으로 
## 직접적인 학습에 사용하지 않고 새로운 데이터에서도 좋은 성능을 내는지 확인하기 위해 쓰입니다.

Valid_Data = pd.read_csv('/content/drive/MyDrive/ToBigs/첫 데이콘/Data/val.csv')
Valid_Data = Valid_Data.drop(columns=['ID'])
Valid_Data.head()

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,V22,V23,V24,V25,V26,V27,V28,V29,V30,Class
0,-0.338262,1.119593,1.044367,-0.222187,0.499361,-0.246761,0.651583,0.069539,-0.736727,-0.366846,...,-0.633753,-0.120794,-0.385050,-0.069733,0.094199,0.246219,0.083076,-0.255991,-0.994878,0
1,0.962496,0.328461,-0.171479,2.109204,1.129566,1.696038,0.107712,0.521502,-1.191311,0.724396,...,0.402492,-0.048508,-1.371866,0.390814,0.199964,0.016371,-0.014605,0.168937,-0.994784,0
2,1.145524,0.575068,0.194008,2.598192,-0.092210,-1.044430,0.531588,-0.241888,-0.896287,0.757952,...,-0.119703,-0.076510,0.691320,0.633984,0.048741,-0.053192,0.016251,0.169496,-0.994502,0
3,0.927060,-0.323684,0.387585,0.544474,0.246787,1.650358,-0.427576,0.615371,0.226278,-0.225495,...,0.079359,0.096632,-0.992569,0.085096,0.377447,0.036096,-0.005960,0.331307,-0.994467,0
4,-3.005237,2.600138,1.483691,-2.418473,0.306326,-0.824575,2.065426,-1.829347,4.009259,6.051521,...,-0.181268,-0.163747,0.515821,0.136318,0.460054,-0.251259,-1.105751,-0.287012,-0.994373,0


In [49]:
## Class column 이 Y value 인데 이를 제외한 나머지 column 을 scaling 하려고 합니다.
## 그러기 위해서 Columns 이라는 list 에 나머지 column 들의 이름을 저장해둡니다.

Columns = []
for i in range(1,31):
    Columns.append('V'+str(i))

In [50]:
## Columns 을 이용해 scaling 을 진행합니다.

from sklearn.preprocessing import StandardScaler
SS = StandardScaler()
SS.fit(Train_Data)

Train_Data[Columns] = SS.transform(Train_Data[Columns])
Train_Data[Columns] = pd.DataFrame(Train_Data[Columns])
Valid_Data[Columns] = SS.transform(Valid_Data[Columns])
Valid_Data[Columns] = pd.DataFrame(Valid_Data[Columns])

Valid_Data.head()

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,V22,V23,V24,V25,V26,V27,V28,V29,V30,Class
0,-0.173475,0.677327,0.691190,-0.154335,0.364809,-0.189318,0.536344,0.060725,-0.670635,-0.347282,...,-0.875560,-0.187839,-0.634092,-0.137315,0.196459,0.616898,0.234701,-0.346696,-1.990670,0
1,0.493220,0.198159,-0.121047,1.496059,0.825644,1.270800,0.084662,0.441968,-1.085624,0.671301,...,0.556061,-0.074193,-2.262901,0.748237,0.416355,0.041562,-0.039585,-0.222190,-1.990502,0
2,0.587031,0.347522,0.123114,1.842214,-0.067776,-0.788809,0.436688,-0.201972,-0.816297,0.702623,...,-0.165376,-0.118216,1.142531,1.215812,0.101948,-0.132560,0.047060,-0.222026,-1.989997,0
3,0.475058,-0.196827,0.252432,0.388385,0.180115,1.236469,-0.359890,0.521149,0.208493,-0.215344,...,0.109638,0.153994,-1.636844,0.160394,0.785362,0.090938,-0.015310,-0.174615,-1.989933,0
4,-1.540417,1.574053,0.984678,-1.709088,0.223653,-0.623576,1.710528,-1.541035,3.661975,5.643730,...,-0.250432,-0.255368,0.852857,0.258886,0.957110,-0.628342,-3.103499,-0.355785,-1.989765,0


In [51]:
class Custom_Dataset(Dataset):
    def __init__(self, data, eval_mode):  
        # 원래 이런 eval_mode 잘 없는데 train 은 y 가 없고 valid 은 y 가 있는 점 때문에 필요한 장치입니다.
        # 원래는 eval_mode 없이 그냥 train valid 모두 다 y 가 있었으면 아래에 전부 다 iter enumerate 쓰면 됩니다.
        self.Data = data
        self.Eval_Mode = eval_mode
        if self.Eval_Mode:
            self.labels = self.Data['Class'].values
            self.Data = self.Data.drop(columns=['Class']).values
        else:
            self.Data = self.Data.values

    def __getitem__(self,index):
        if self.Eval_Mode:
            self.x = torch.Tensor(self.Data[index])
            self.y = self.labels[index]
            return self.x,self.y
        else:
            return torch.Tensor(self.Data[index])

    def __len__(self):
        return len(self.Data)

In [52]:
## 모델 설계하는 부분입니다.

## 처음 시작할때는 weighted XGBoost 알고리즘을 사용하다가 성능에 한계가 보여서 간단한 딥러닝 모델을 구현하려고 했습니다.
## 이때 다른 사이트 참고하면서 임의로 만들었지만 성능이 XGBoost 보다는 좋았지만 기대 이하의 성능을 보였습니다.
## 계속 고민하다 코드공유에 아래 코드의 출처가 올라왔고 매우 좋은 성능을 보였다 ㅎ;;
## 그 코드 기반으로 조금 더 수정하고 보완을 해 최종 결과를 냈습니다.

class AE(nn.Module):
    def __init__(self):
        super(AE,self).__init__()
        self.Encoder = nn.Sequential(
            nn.Linear(30,64),
            nn.BatchNorm1d(64),
            nn.ReLU(),

            nn.Linear(64,128),
            nn.BatchNorm1d(128),
            nn.ReLU(),

            # nn.Linear(128,256),
            # nn.BatchNorm1d(256),
            # nn.ReLU(),

            # nn.Linear(128,64),
            # nn.BatchNorm1d(64),
            # nn.ReLU(),

            # nn.Linear(64,16),
            # nn.BatchNorm1d(16),
            # nn.ReLU(),
        )

        self.Decoder = nn.Sequential(
            # nn.Linear(256,128),
            # nn.BatchNorm1d(128),
            # nn.ReLU(),

            # nn.Linear(16,64),
            # nn.BatchNorm1d(64),
            # nn.ReLU(),

            # nn.Linear(64,128),
            # nn.BatchNorm1d(128),
            # nn.ReLU(),

            nn.Linear(128,64),
            # nn.BatchNorm1d(64),
            nn.ReLU(),

            nn.Linear(64,30),
        )

    def forward(self,x):
        x = self.Encoder(x)
        x = self.Decoder(x)
        return x

In [53]:
## 파라미터 설정.

epochs = 150
batch_size = 1024
lr = 0.0003
Dist_eps = 0.0000008
alpha_range = 0.95
Valid_Times = 20


In [54]:
## 데이터셋을 mini-batch stochastic learning 에 사용할 수 있게 1024 크기의 batch 으로 만듭니다.

Custom_Train_Data = Custom_Dataset(Train_Data,eval_mode = False)
train_loader = DataLoader(Custom_Train_Data, batch_size = batch_size, shuffle=True, num_workers=2)
Custom_Valid_Data = Custom_Dataset(Valid_Data,eval_mode = True)
valid_loader = DataLoader(Custom_Valid_Data, batch_size = batch_size, shuffle=True, num_workers=2)

train_loader

In [55]:
## 모델을 선언하고 학습을 시킵니다.

AEmodel = nn.DataParallel(AE())
AEmodel.eval()
optimizer = torch.optim.Adam(params = AEmodel.parameters(), lr = lr)
cos = nn.CosineSimilarity(dim=1, eps=Dist_eps).to(device)
criterion = nn.L1Loss().to(device)

score=0
for epoch in range(epochs):
    AEmodel.train()
    train_loss=[]
    for data_x in iter(train_loader):

        data_x = data_x.float().to(device)
        pred_x = AEmodel(data_x)
        loss = criterion(data_x,pred_x)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        train_loss.append(loss.item())

    if (epoch+1)%(int(epochs/Valid_Times))==0:
        AEmodel.eval()
        pred = []
        true = []
        with torch.no_grad():
            for data_x, label in iter(valid_loader):
                data_x = data_x.float().to(device)

                pred_x = AEmodel(data_x)
                diff = cos(data_x, pred_x).cpu().tolist()
                batch_pred = np.where(np.array(diff)<alpha_range, 1,0).tolist()
                pred += batch_pred
                true += label.tolist()

        f1score = f1_score(true, pred, average='macro')
        print("Epoch : [{} / {}], Train Loss : {:.4f}, Valid F1Score : {:.4f}".format(epoch+1,epochs,np.mean(train_loss),f1score))


Epoch : [7 / 150], Train Loss : 0.1410, Valid F1Score : 0.4871
Epoch : [14 / 150], Train Loss : 0.1090, Valid F1Score : 0.5067
Epoch : [21 / 150], Train Loss : 0.0915, Valid F1Score : 0.5387
Epoch : [28 / 150], Train Loss : 0.0838, Valid F1Score : 0.5407
Epoch : [35 / 150], Train Loss : 0.0766, Valid F1Score : 0.6993
Epoch : [42 / 150], Train Loss : 0.0708, Valid F1Score : 0.8376
Epoch : [49 / 150], Train Loss : 0.0669, Valid F1Score : 0.8470
Epoch : [56 / 150], Train Loss : 0.0651, Valid F1Score : 0.8622
Epoch : [63 / 150], Train Loss : 0.0620, Valid F1Score : 0.8786
Epoch : [70 / 150], Train Loss : 0.0610, Valid F1Score : 0.8845
Epoch : [77 / 150], Train Loss : 0.0581, Valid F1Score : 0.8845
Epoch : [84 / 150], Train Loss : 0.0560, Valid F1Score : 0.8905
Epoch : [91 / 150], Train Loss : 0.0571, Valid F1Score : 0.8845
Epoch : [98 / 150], Train Loss : 0.0557, Valid F1Score : 0.8967
Epoch : [105 / 150], Train Loss : 0.0543, Valid F1Score : 0.9031
Epoch : [112 / 150], Train Loss : 0.0525

In [56]:
## 예측 함수를 만듭니다.

def Predict(model, x_data_loaded):
    model.eval()
    pred=[]
    with torch.no_grad():
        for data_x in iter(x_data_loaded):
            data_x = data_x.float().to(device)
            pred_x = AEmodel(data_x)
            diff = cos(data_x,pred_x).cpu().tolist()
            batch_pred = np.where(np.array(diff)<alpha_range,1,0).tolist()
            pred+=batch_pred
    return pred

In [57]:
## Imbalanced Dataset 인 만큼 Accuracy 보다 f1score 이 훨씬 중요한 척도입니다.
## f1score 을 구하는 함수를 만듭니다.

def f1score(Real,Pred) :
    Pred = np.array(Pred)
    y = np.array(Real)
    TP = np.sum((Pred == 1) & (y == 1))
    FP = np.sum((Pred == 1) & (y != 1))
    FN = np.sum((Pred != 1) & (y == 1))
    TN = np.sum((Pred != 1) & (y != 1))
    Precision = (TP)/(TP+FP)
    Recall = (TP)/(TP+FN)
    score = 2*(Precision*Recall)/(Precision+Recall)
    return score

In [58]:
## Valid data 을 통해 새로운 데이터에서도 유사한 성능을 보이는지 확인합니다.

new_Valid_Data = pd.read_csv('/content/drive/MyDrive/ToBigs/첫 데이콘/Data/val.csv')

valid_real = new_Valid_Data['Class']
new_Valid_Data = new_Valid_Data.drop(columns=['ID','Class'])

new_Valid_Data[Columns] = SS.transform(new_Valid_Data[Columns])
new_Valid_Data[Columns] = pd.DataFrame(new_Valid_Data[Columns])
Custom_new_Valid_Data = Custom_Dataset(new_Valid_Data,eval_mode = False) 

new_valid_loader = DataLoader(Custom_new_Valid_Data,batch_size = batch_size, shuffle=False,num_workers=2)
valid_pred = Predict(AEmodel,new_valid_loader)
valid_f1score = f1score(valid_real,valid_pred)
print(f1_score(valid_real,valid_pred))

0.8064516129032259


In [59]:
print(valid_f1score)

0.8064516129032259


In [60]:
print("Valid Data F1 Score : ", valid_f1score)
#   0.83333

Valid Data F1 Score :  0.8064516129032259


In [61]:
test_df = pd.read_csv('/content/drive/MyDrive/ToBigs/첫 데이콘/Data/test.csv')
test_df = test_df.drop(columns=['ID'])
test_df[Columns] = SS.transform(test_df[Columns])
test_df[Columns] = pd.DataFrame(test_df[Columns])
custom_test_df = Custom_Dataset(test_df,eval_mode = False)
test_loader = DataLoader(custom_test_df,batch_size = batch_size,shuffle=False,num_workers=2)
test_pred = Predict(AEmodel,test_loader)

In [62]:
test_pred

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,


In [63]:
# submit = pd.read_csv('/content/drive/MyDrive/ToBigs/첫 데이콘/Data/sample_submission.csv')
# print(submit.columns)
# submit.head()

In [64]:
# submit['Class']=test_pred
# submit.to_csv('submit.csv')

In [65]:
# submit.head()